In [9]:
import config
from dataloader import iMetDataset
import pandas as pd

In [ ]:
conf = config.Config()
train_df = pd.read_csv(conf.DATA_PATH+'/train.csv')

In [3]:
d = iMetDataset(conf, train_df, get_transform(train=True))

NameError: name 'get_transform' is not defined

In [6]:
d[0]

(3, 5214, 3676)


KeyboardInterrupt: 

# Model

In [1]:
import config
from dataloader import iMetDataset
import pandas as pd

import torch
import torchvision
import torch.nn as nn
import copy
import utils
from utils import get_transform

from engine import train_one_epoch, evaluate
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [2]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    print(in_features)
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    print('input fo mask classifier', in_features_mask)
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,hidden_layer,num_classes)

    return model



In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

conf = config.Config()
train_df = pd.read_csv(conf.DATA_PATH+'/train.csv')
print('Loaded train_df!')
print(train_df.head())

Loaded train_df!
                            ImageId  \
0  00000663ed1ff0c4e0132b9b9ac53f6e   
1  00000663ed1ff0c4e0132b9b9ac53f6e   
2  00000663ed1ff0c4e0132b9b9ac53f6e   
3  00000663ed1ff0c4e0132b9b9ac53f6e   
4  00000663ed1ff0c4e0132b9b9ac53f6e   

                                       EncodedPixels  Height  Width  ClassId  \
0  6068157 7 6073371 20 6078584 34 6083797 48 608...    5214   3676        6   
1  6323163 11 6328356 32 6333549 53 6338742 75 63...    5214   3676        0   
2  8521389 10 8526585 30 8531789 42 8537002 46 85...    5214   3676       28   
3  12903854 2 12909064 7 12914275 10 12919485 15 ...    5214   3676       31   
4  10837337 5 10842542 14 10847746 24 10852951 33...    5214   3676       32   

                     AttributesIds  
0  115,136,143,154,230,295,316,317  
1  115,136,142,146,225,295,316,317  
2                              163  
3                          160,204  
4                              219  


In [4]:
torch.random.manual_seed(0)
d = iMetDataset(conf, train_df, get_transform(train=True))
data_loader = torch.utils.data.DataLoader(d, batch_size=conf.BATCH_SIZE, shuffle=True, num_workers=2,collate_fn=utils.collate_fn)
print('Initialized data loader')

# get the model using our helper function
model = get_model_instance_segmentation(conf.NO_OF_CLASSES)
print('Loaded model!')
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3, gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10



Initialized data loader
1024
input fo mask classifier 256
Loaded model!


In [ ]:
for epoch in range(num_epochs):
    print('Epoch '+str(epoch))
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    print("Evaluating at end of epoch")
    evaluate(model, data_loader_test, device=device)

print("That's it!")


Epoch 0
